In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')

In [87]:
testingSet = pd.read_csv('resources/test_final_100k.csv')

# Reducimos a algo minimo para probar algoritmos

In [88]:
postulacionesSample = postulaciones.sample(frac=0.00001)

In [12]:
postulacionesSample.info

<bound method DataFrame.info of             idaviso idpostulante     fechapostulacion
548054   1112285953      96XzlN8  2018-02-01 05:19:20
2643033  1112369058      owj6dPk  2018-03-09 08:31:25
3562866  1112448632      BmBxmjR  2018-04-06 20:07:15
4648591  1111930643      qevrEJb  2018-04-08 11:55:08
2862330  1112368514      8MBXGNx  2018-03-11 10:27:35
4744422  1112266469      3NlpAkx  2018-03-27 08:33:17
1229441  1111972061      kPNwRQV  2018-01-29 10:48:20
1675671  1112372532      BmaZz1R  2018-03-01 20:00:07
1667449  1112269370       64lQDl  2018-01-22 18:14:03
706997   1112458518      0zB0YGv  2018-04-11 18:44:10
952941   1112252543      KBrG5oA  2018-01-19 03:32:18
4593251  1112312923      W9A1PAe  2018-02-14 09:29:07
188143   1112403375      GNZq2YK  2018-03-15 18:45:51
3965957  1112313265       5wP1vw  2018-02-07 02:51:52
2747291  1112170115      3NQmDoJ  2018-03-15 12:07:58
4038301  1112418593       eApZ12  2018-03-22 22:15:42
668599   1112396657      A3XP5El  2018-03-13 19:11

# Categoricals a Numericals

## Postulantes Genero

In [89]:
import datetime as dt

postulantesGenero['fechanacimiento'] = pd.to_datetime(postulantesGenero['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
postulantesGenero = postulantesGenero.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
postulantesGenero['edad'] = (hoy - postulantesGenero['fechanacimiento']).astype('<m8[Y]')

In [90]:
postulantesGeneroOHC = pd.concat([postulantesGenero,pd.get_dummies(postulantesGenero['sexo'], prefix='sexo')],axis=1)

postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','fechanacimiento','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

## Educacion 

In [92]:
postulantesEducacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['estado'], prefix='estado')],axis=1)
postulantesEducacionOHC = pd.concat([postulantesEducacionOHC,pd.get_dummies(postulantesEducacion['nombre'], prefix='nombre')],axis=1)

postulantesEducacionOHC.drop(['nombre'],axis=1, inplace=True)
postulantesEducacionOHC.drop(['estado'],axis=1, inplace=True)

## Detalle 

In [125]:
avisosDetalleFiltered = avisosDetalle.loc[avisosDetalle['nombre_zona'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nombre_area'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['tipo_de_trabajo'].notnull(),:]
avisosDetalleFiltered = avisosDetalleFiltered.loc[avisosDetalleFiltered['nivel_laboral'].notnull(),:]

In [126]:
avisosDetalleFiltered['denominacion_empresa'].fillna('S/E', inplace=True)

In [127]:
avisosDetalleOHC = pd.concat([avisosDetalleFiltered,pd.get_dummies(avisosDetalleFiltered['nombre_zona'], prefix='zona')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['tipo_de_trabajo'], prefix='tipo')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['nivel_laboral'], prefix='nivel')],axis=1)
avisosDetalleOHC = pd.concat([avisosDetalleOHC,pd.get_dummies(avisosDetalleFiltered['nombre_area'], prefix='area')],axis=1)

In [128]:
avisosDetalleOHC.drop(['ciudad','mapacalle'],axis=1,inplace=True)

# Mergeo con Postulaciones

In [129]:
postulacionesConAvisos = pd.merge(avisosDetalleOHC,postulaciones, how='inner',on='idaviso')

In [ ]:
postulacionesConAvisos['cantidadEmpresa'] = 1;
cantAvisosPorEmpresa = postulacionesConAvisos.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]

In [124]:
cantAvisosPorEmpresa.head()

,denominacion_empresa,cantidadEmpresa
0,AB InBev - Cervecería y Maltería Quilmes,1
1,ABAC S R L,1
2,ALESSANDRINI S R L,1
3,Adecco - Región NORTE & OESTE GBA,1
4,Adecco -Región GBA SUR,1


In [ ]:
avisosDetalleOHC = pd.merge(avisosDetalleOHC,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')

In [ ]:
avisosDetalleOHC.drop('denominacion_empresa',inplace=True,axis=1)
#Considero a la empresa como cantidad, esto agruparia tambien a las que son iguales

In [115]:
avisosDetalleOHC

,idaviso,idpostulante,idpais,titulo,descripcion,zona_Capital Federal,zona_Gran Buenos Aires,tipo_Full-time,tipo_Part-time,nivel_Junior,...,area_Oficios y Profesiones,area_Planeamiento comercial,area_Producción,area_Recepcionista,area_Secretaria,area_Seguridad,area_Telemarketing,area_Tesorería,area_Ventas,cantidadEmpresa
0,1112285953,96XzlN8,1,Representante de Atención al cliente,"<p align=""center"">EN <strong>MO&amp;PC</strong...",0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2364
1,1112369058,owj6dPk,1,Ayudante de Contador,Ayudante de Contador para Estudio Contable zon...,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,133
2,1112448632,BmBxmjR,1,SUPERVISOR DE LOGISTICA,<p>&lt;p&lt;p</p><p>Importante Empresa de Reta...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,397
3,1111930643,qevrEJb,1,Mucamas/os para importante Hotel 5*,<p><strong> Si te g...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3278
4,1112342970,YjQdb6V,1,Cadete,"<p style=""""><strong>Adecco Office</strong> est...",0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,3278
5,1112368514,8MBXGNx,1,Coordinador de contrataciones de proveedores l...,<p>Importante empresa de Logística multinacion...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,143
6,1111972061,kPNwRQV,1,Cajero Bancario - eventual verano (Sur GBA),<p><strong>Banco Columbia </strong>se encuentr...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,893
7,1112372532,BmaZz1R,1,Cadete de Deposito en San Fernando,<p>En <strong>Garbarino </strong>nos encontram...,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,532
8,1112269370,64lQDl,1,S- Contador,<p>Importante empresa de recoleccion de residu...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,67
9,1112458518,0zB0YGv,1,Asesor de atencion al cliente y caja,<p>Importante cadena multinacional de supermer...,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1933


In [ ]:
avisosDetalleOHC.drop(['nombre_zona'],axis=1, inplace=True)
avisosDetalleOHC.drop(['tipo_de_trabajo'],axis=1, inplace=True)
avisosDetalleOHC.drop(['nivel_laboral'],axis=1, inplace=True)
avisosDetalleOHC.drop(['nombre_area'],axis=1, inplace=True)

# Postulados con Sample

In [113]:
avi

## Creamos avisos no postulados 

In [65]:
#avisosVistos = pd.merge(postulacionesSample,vistas,left_on=['idpostulante','idaviso'],right_on=['idpostulante','idAviso'],how='inner')

In [74]:
#avisosNoPostulados = pd.merge(postulacionesSample,vistas,left_on='idpostulante',right_on='idpostulante',how='right')

In [75]:
#avisosNO = avisosNoPostulados[avisosNoPostulados['idaviso'].isnull()]

In [112]:
##avisosNO.sample()

In [111]:
#avisosNoVistos.sample(frac=0.00001)

In [80]:
#avisosNoVistos = avisosNoVistos.drop(['fechapostulacion','idaviso','timestamp'],axis=1)

In [82]:
#avisosNoVistos = avisosNoVistos[avisosNoVistos['idAviso'].notnull()]

In [84]:
# Load libraries
import pandas
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor